<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LlamaIndex - Integrate with Neo4j

<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LlamaIndex/LlamaIndex_Integrate_with_Neo4j.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/>
</a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=LlamaIndex+-+Integrate+with+Neo4j:+Error+short+description">Bug report</a>



**Tags:** #llamaindex #neo4j #integration #graphstore #request #response

**Author:** [Mahimai Raja J](http://github.com/mahimairaja)

**Last update:** 2023-10-15 (Created: 2023-10-08)

**Description:** This notebook demonstrates how to use Llamaindex and Neo4j as graph store to perform request and response model for the custom data. It is usefull for organizations to quickly integrate Neo4j with Llamaindex.

**References:**
- [Llamaindex Documentation](https://docs.llamaindex.ai/en/stable/examples/index_structs/knowledge_graph/Neo4jKGIndexDemo.html)
- [Neo4j Documentation](https://neo4j.com/docs/)

![img](assets/rag.jpg)

## Input

> Exlore other [data connectors](https://docs.llamaindex.ai/en/stable/core_modules/data_modules/connector/modules.html)

In [ ]:
data_path = "data"

### Import libraries

In [ ]:
try:
    import llama_index
except ModuleNotFoundError:
    !pip install -q llama-index
    import llama_index
try:
    import neo4j
except ModuleNotFoundError:
    !pip install neo4j --user
    import neo4j

import os
import logging
import sys
from llama_index.llms import OpenAI
from llama_index import ServiceContext
from llama_index import (
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
    SimpleDirectoryReader,
)
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import Neo4jGraphStore


from llama_index.llms import OpenAI
from IPython.display import Markdown, display

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

### Setup variables
- `neo4j_url`: URL of the Neo4j instance
- `neo4j_username`: Username of the Neo4j instance
- `neo4j_password`: Password of the Neo4j instance
- `neo4j_graph`: Graph name of the Neo4j instance

> Create your neo4j [AuroDS instance](https://console.neo4j.io/?product=aura-ds) or [Sandbox](https://sandbox.neo4j.com/)

In [ ]:
neo4j_url = "http://localhost:7474"
neo4j_username = "neo4j"
neo4j_password = "password"
neo4j_graph = "graph"

## Model

### Setup environ

In [ ]:
os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

documents = SimpleDirectoryReader(
    data_path
).load_data()

### Create Neo4j instance

Building the Knowledge Graph using `Neo4jGraphStore`

In [ ]:
graph_store = Neo4jGraphStore(
    username=neo4j_username,
    password=neo4j_password,
    url=neo4j_url,
    database=neo4j_graph,
)

### Create Llamaindex instance

Instantiate Neo4jGraph `Knowledge Graph Indexes`

![img](assets/indexing.jpg)

In [ ]:
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
    service_context=service_context,
)

In [ ]:
input_query = input("Enter the query to perform retrieval\n")

In [ ]:
query_engine = index.as_query_engine(include_text=True, response_mode="tree_summarize")

response = query_engine.query(input_query)

## Output

### Display result

In [ ]:
display(Markdown(f"<b>{response}</b>"))